House Pricing in Leeds project 
Webscrapping, Exploratory data analysis, data modeling for price predicting

In [3]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

main_list = []

for n in range(40):
    link = "https://www.rightmove.co.uk/house-prices/leeds.html?soldIn=1&page=" + str(n+1)
    response = requests.get(link)
    content = response.text
    soup = BeautifulSoup(content, "html.parser")
    script = str(soup.find_all(name="script")[1]).replace("<script>", "").replace("</script>", "").replace("window.__PRELOADED_STATE__ = ","")
    json_str = json.loads(script)
    new_dict = json_str["results"]
    result = new_dict["properties"]
    for item in result:
        main_list.append(item)
        
    print("completed page "+str(n+1))  #Pint confirming each page has gone through ok


completed page 1
completed page 2
completed page 3
completed page 4
completed page 5
completed page 6
completed page 7
completed page 8
completed page 9
completed page 10
completed page 11
completed page 12
completed page 13
completed page 14
completed page 15
completed page 16
completed page 17
completed page 18
completed page 19
completed page 20
completed page 21
completed page 22
completed page 23
completed page 24
completed page 25
completed page 26
completed page 27
completed page 28
completed page 29
completed page 30
completed page 31
completed page 32
completed page 33
completed page 34
completed page 35
completed page 36
completed page 37
completed page 38
completed page 39
completed page 40


In [4]:
df = pd.DataFrame(main_list)
df.head()

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl
0,"6, Newton Court, Leeds, West Yorkshire LS8 2PH",Flat,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,False,"[{'displayPrice': '£155,000', 'dateSold': '23 ...","{'lat': 53.82443, 'lng': -1.49622}",https://www.rightmove.co.uk/house-prices/detai...
1,"6, Maple Court, Seacroft, Leeds, West Yorkshir...",Flat,NaN,{'imageUrl': '/spw/images/placeholder/no-image...,False,"[{'displayPrice': '£127,000', 'dateSold': '19 ...","{'lat': 53.8088, 'lng': -1.47742}",
2,"24, Pinfold Road, Leeds, West Yorkshire LS15 0PN",Semi-Detached,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£167,500', 'dateSold': '16 ...","{'lat': 53.79587, 'lng': -1.45936}",https://www.rightmove.co.uk/house-prices/detai...
3,"5, Thorpe Garth, Leeds, West Yorkshire LS10 4QD",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£144,000', 'dateSold': '16 ...","{'lat': 53.7389, 'lng': -1.54924}",https://www.rightmove.co.uk/house-prices/detai...
4,"16, Laurel Hill Gardens, Leeds, West Yorkshire...",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£256,000', 'dateSold': '11 ...","{'lat': 53.79238, 'lng': -1.44317}",https://www.rightmove.co.uk/house-prices/detai...


In [5]:
df.drop('detailUrl', inplace=True, axis=1)
df.drop('images', inplace=True, axis=1)
df.drop('hasFloorPlan', inplace=True, axis=1)
df.drop('location', inplace=True, axis=1)
series_transactions = [n['transactions'][0]['displayPrice'] for n in main_list]
series_num_of_transactions = [len(n['transactions']) for n in main_list]
series_newBuild = [n['transactions'][0]['newBuild'] for n in main_list]
series_dateSold = [n['transactions'][0]['dateSold'] for n in main_list]
df['prices'] = series_transactions
df['new'] = series_newBuild
df['dateSold'] = series_dateSold
df['num_of_transactions'] = series_num_of_transactions
df['postcode'] = df['address'].str[-8:]
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address              1000 non-null   object 
 1   propertyType         1000 non-null   object 
 2   bedrooms             758 non-null    float64
 3   transactions         1000 non-null   object 
 4   prices               1000 non-null   object 
 5   new                  1000 non-null   bool   
 6   dateSold             1000 non-null   object 
 7   num_of_transactions  1000 non-null   int64  
 8   postcode             1000 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 63.6+ KB


['address',
 'propertyType',
 'bedrooms',
 'transactions',
 'prices',
 'new',
 'dateSold',
 'num_of_transactions',
 'postcode']

In [6]:
df.head()

,address,propertyType,bedrooms,transactions,prices,new,dateSold,num_of_transactions,postcode
0,"6, Newton Court, Leeds, West Yorkshire LS8 2PH",Flat,3.0,"[{'displayPrice': '£155,000', 'dateSold': '23 ...","£155,000",False,23 Aug 2021,2,LS8 2PH
1,"6, Maple Court, Seacroft, Leeds, West Yorkshir...",Flat,NaN,"[{'displayPrice': '£127,000', 'dateSold': '19 ...","£127,000",False,19 Aug 2021,2,LS14 6FS
2,"24, Pinfold Road, Leeds, West Yorkshire LS15 0PN",Semi-Detached,3.0,"[{'displayPrice': '£167,500', 'dateSold': '16 ...","£167,500",False,16 Aug 2021,1,LS15 0PN
3,"5, Thorpe Garth, Leeds, West Yorkshire LS10 4QD",Terraced,3.0,"[{'displayPrice': '£144,000', 'dateSold': '16 ...","£144,000",False,16 Aug 2021,6,LS10 4QD
4,"16, Laurel Hill Gardens, Leeds, West Yorkshire...",Terraced,3.0,"[{'displayPrice': '£256,000', 'dateSold': '11 ...","£256,000",False,11 Aug 2021,4,LS15 9EJ


In [7]:
df.tail()

,address,propertyType,bedrooms,transactions,prices,new,dateSold,num_of_transactions,postcode
995,"46, St Helens Lane, Adel, Leeds, West Yorkshir...",Detached,5.0,"[{'displayPrice': '£572,240', 'dateSold': '20 ...","£572,240",False,20 Mar 2021,1,LS16 8BS
996,"68, Cliff Road, Leeds, West Yorkshire LS6 2EZ",Flat,2.0,"[{'displayPrice': '£185,950', 'dateSold': '19 ...","£185,950",False,19 Mar 2021,3,LS6 2EZ
997,"18, Hill End Crescent, Leeds, West Yorkshire L...",Semi-Detached,1.0,"[{'displayPrice': '£255,000', 'dateSold': '19 ...","£255,000",False,19 Mar 2021,2,LS12 3PW
998,"8, Hough Terrace, Leeds, West Yorkshire LS13 3QT",Terraced,2.0,"[{'displayPrice': '£125,000', 'dateSold': '19 ...","£125,000",False,19 Mar 2021,4,LS13 3QT
999,"6, Birkdale Green, Leeds, West Yorkshire LS17 7SP",Semi-Detached,3.0,"[{'displayPrice': '£275,000', 'dateSold': '19 ...","£275,000",False,19 Mar 2021,1,LS17 7SP


In [8]:
#format dateSold
#format prices to int
df['bedrooms'] = df['bedrooms'].fillna(0).astype(int)
df["prices"] = df['prices'].str.replace(",","").str.replace("£","")
df["dateSold"] = df["dateSold"].str.replace("Jan","/1/").str.replace("Feb","/2/").str.replace("Mar","/3/").str.replace("Apr","/4/").str.replace("May","/5/").str.replace("Jun","/6/").str.replace("Jul","/7/").str.replace("Aug","/8/").str.replace("Sep","/9/").str.replace("Oct","/10/").str.replace("Nov","/11/").str.replace("Dec","/12/").str.replace(" ","")
df["dateSold"] = pd.to_datetime(df["dateSold"])
df["prices"] = df["prices"].astype(int)
df.drop('transactions', inplace=True, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   address              1000 non-null   object        
 1   propertyType         1000 non-null   object        
 2   bedrooms             1000 non-null   int32         
 3   prices               1000 non-null   int32         
 4   new                  1000 non-null   bool          
 5   dateSold             1000 non-null   datetime64[ns]
 6   num_of_transactions  1000 non-null   int64         
 7   postcode             1000 non-null   object        
dtypes: bool(1), datetime64[ns](1), int32(2), int64(1), object(3)
memory usage: 48.0+ KB


In [9]:
property = df['propertyType'].unique()
print(property)

['Flat' 'Semi-Detached' 'Terraced' 'Detached']


In [10]:
import numpy as np
# ['Semi-Detached' 'Flat' 'Detached' 'Terraced']
# create a list of our conditions
conditions = [
    (df['propertyType'] == 'Semi-Detached'),
    (df['propertyType'] == 'Detached'),
    (df['propertyType'] == 'Terraced'),
    (df['propertyType'] == 'Flat')
    ]

# create a list of the values we want to assign for each condition
values = [1,2,3,4]

# create a new column and use np.select to assign values to it using our lists as arguments
df['prop_type_num'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,address,propertyType,bedrooms,prices,new,dateSold,num_of_transactions,postcode,prop_type_num
0,"6, Newton Court, Leeds, West Yorkshire LS8 2PH",Flat,3,155000,False,2021-08-23,2,LS8 2PH,4
1,"6, Maple Court, Seacroft, Leeds, West Yorkshir...",Flat,0,127000,False,2021-08-19,2,LS14 6FS,4
2,"24, Pinfold Road, Leeds, West Yorkshire LS15 0PN",Semi-Detached,3,167500,False,2021-08-16,1,LS15 0PN,1
3,"5, Thorpe Garth, Leeds, West Yorkshire LS10 4QD",Terraced,3,144000,False,2021-08-16,6,LS10 4QD,3
4,"16, Laurel Hill Gardens, Leeds, West Yorkshire...",Terraced,3,256000,False,2021-11-08,4,LS15 9EJ,3


In [11]:
df = pd.concat([df, pd.get_dummies(df['propertyType'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['new'])], axis=1)
df.head()

,address,propertyType,bedrooms,prices,new,dateSold,num_of_transactions,postcode,prop_type_num,Detached,Flat,Semi-Detached,Terraced,False
0,"6, Newton Court, Leeds, West Yorkshire LS8 2PH",Flat,3,155000,False,2021-08-23,2,LS8 2PH,4,0,1,0,0,1
1,"6, Maple Court, Seacroft, Leeds, West Yorkshir...",Flat,0,127000,False,2021-08-19,2,LS14 6FS,4,0,1,0,0,1
2,"24, Pinfold Road, Leeds, West Yorkshire LS15 0PN",Semi-Detached,3,167500,False,2021-08-16,1,LS15 0PN,1,0,0,1,0,1
3,"5, Thorpe Garth, Leeds, West Yorkshire LS10 4QD",Terraced,3,144000,False,2021-08-16,6,LS10 4QD,3,0,0,0,1,1
4,"16, Laurel Hill Gardens, Leeds, West Yorkshire...",Terraced,3,256000,False,2021-11-08,4,LS15 9EJ,3,0,0,0,1,1


In [23]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

X = df [['bedrooms']]
Y = df ['prices']

lm.fit(X,Y)
Yhat = lm.predict(X)
print(lm.intercept_)
print(lm.coef_)
b_zero = lm.intercept_
b_one = lm.coef_
# estimated_price considering only 'bedrooms' as independent variable
df['estimated_price'] = b_zero - b_one*df['bedrooms']
df.head()
Yhat2 = lm2.predict(X)

b_zero2 = lm2.intercept_
b_one2 = lm2.coef_
# estimated_price considering only 'bedrooms' as independent variable  
df['estimated_price2'] = b_zero2 - b_one2*df['prop_type_num']
df.head()

LinearRegression()